In [3]:
import requests
import time 

PAGE_URL_LIST = [
    'http://example.com/1.page',
    'http://example.com/2.page',
    'http://example.com/3.page',
]

for page_url in PAGE_URL_LIST:
    res = requests.get(page_url,timeout=30)
    print(
        '페이지 URL: {}, HTP status:{},  처리 시간(초):{}'.format(
            page_url,
            res.status_code,
            res.elapsed.total_seconds()
        )
    )

페이지 URL: http://example.com/1.page, HTP status:404,  처리 시간(초):0.552758
페이지 URL: http://example.com/2.page, HTP status:404,  처리 시간(초):0.517445
페이지 URL: http://example.com/3.page, HTP status:404,  처리 시간(초):0.50406


2

디버그 전용 로그와 오류 전용 로그를 다른 파일에 출력하면서 터미널의 포준 출력에도 로그를 출력하려면 바로 아래처럼 구현해야합니다. 

In [10]:
import json
import time
import requests

PAGE_URL_LIST = [
    'http://example.com/1.page',
    'http://example.com/2.page',
    'http://example.com/3.page',
]

def fetch_pages():
    '''페이지의 내용을 추출합니다.'''
    # 처리기록 전용 로그 파일을 append 모드로 엽니다.
    f_info_log = open('crawler_info.log','a', encoding='utf-8')

    # 오류기록 전용 로그 파일을 append mode로 엽니다.
    f_error_log = open('crawler_error.log','a', encoding='utf-8')

    # 추출 내용을 저장할 딕셔너리
    page_contents = {}

    # 터미널에 처리 시작을 출력하고, 로그 파일에도 메시지를 출력
    msg = '크롤링을 시작합니다.\n'
    print(msg)
    f_info_log.write(msg)
    
    for page_url in PAGE_URL_LIST:
        r = requests.get(page_url, timeout=30)

        try:
            r.raise_for_status() # 응답에 에러 발생시 예외를 발생시킵니다.
        except requests.exceptions.RequestException as e:
            # requests와 관련된 예외가 발생하면
            # 터미널과 오류 로그에 오류를 출력합니다. 
            msg = '[ERROR] {exception}\n'.format(exception=e)
            print(msg)
            f_error_log.write(msg)
            continue # 예외가 발생하면 반복을 중지하는게 아니라 건너 뜁니다.

        # 정상적으로 내용을 추출했다면 딕셔너리에 내용을 저장합니다.page_url
        page_contents[page_url] = r.text
        time.sleep(1)

    f_info_log.close()
    f_error_log.close()

    return page_contents


if __name__ == '__main__':
    page_contents = fetch_pages()
    f_page_contents = open('page_contents.json','w')
    json.dump(page_contents, f_page_contents, ensure_ascii=False)
    f_page_contents.close()


크롤링을 시작합니다.

[ERROR] 404 Client Error: Not Found for url: http://example.com/1.page

[ERROR] 404 Client Error: Not Found for url: http://example.com/2.page

[ERROR] 404 Client Error: Not Found for url: http://example.com/3.page



```
1. 파일 다루기 기본
파이썬에서 파일 다룰 때는 기본 내장함수 open() 함수를 활용합니다.
open() 함수의 사용법
첫번째 인수 file경로만이 필수입니다.
open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)¶
open() 함수 인자, 자주 사용하는 것만 살펴봅니다.

file : 파일 경로
mode : 파일이 열리는 모드
'r' : 읽기 용으로 열림 (기본값)
'w' : 쓰기 위해 열기, 파일을 먼저 자른다.
'x' : 베타적 생성을 위해 열리고, 이미 존재하는 경우 실패
'a' : 쓰기를 위해 열려 있고, 파일의 끝에 추가하는 경우 추가합니다.
'b' : 2진 모드(바이너리 모드)
't' : 텍스트 모드 (기본값)
'+' : 업데이트 (읽기 및 쓰기)를 위한 디스크 파일 열기
'U' : 유니버설 개행 모드 (사용되지 않음)
buffering : 버퍼링끄기는 0(바이너리모드에서만 동작함), 라인모드는 1 (텍스트 모드에서만 가능), 고정 크기로 보내려면 임의의 바이트수를 1보다 큰 양의 수로 입력, 기본 정책은 아래와 같습니다.
이진 파일은 고정 크기 청크로 버퍼링됩니다. 버퍼의 크기는 기본 장치의 "블록 크기"를 결정하고 다시 떨어지는 경험적 방법을 사용하여 선택됩니다 io.DEFAULT_BUFFER_SIZE. 많은 시스템에서 버퍼는 일반적으로 4096 또는 8192 바이트 길이입니다.
"대화식"텍스트 파일 ( isatty() 반환 되는 파일 True)은 회선 버퍼링을 사용합니다. 다른 텍스트 파일은 바이너리 파일에 대해 위에서 설명한 정책을 사용합니다.
encoding : 파일을 디코딩하거나 인코딩하는데 사용되는 이름, 대부분 utf-8 이지만 모든 시스템이 utf-8이라는 보장이 없으므로 명시적으로 하는 것이 좋다.
파이썬에서 지원되는 인코딩 목록 - https://docs.python.org/3.6/library/codecs.html
파일 객체는 반드시 열고 작업이 완료되면 반드시 파일을 닫아야 합니다.

파일을 닫지 않으면 버퍼링되어 있는 데이터는 기록되지 않고 소실될 수 있습니다.
file 객체의 close()메소드로 파일을 닫습니다.
```

## with구문으로 close 메서드 누수 막기

파일 개겣의 close 메서드 누수를 막을 수 있는 기능이 있습니다. 바로 with구문입니다.    
with구문을 사용하면 with 구문 블록을 빠져나올 떄 close 메서드가 자동으로 호출됩니다. 또한 예외가 발생했을 떄도 close 메서드가 호출됩니다.    



In [14]:
import json
import time
import requests

PAGE_URL_LIST = [
    'http://example.com/1.page.html',
    'http://example.com/2.page.html',
    'http://example.com/3.page.html',
]

def fetch_pages():
    '''페이지의 내용을 추출합니다.'''
    # 처리기록 전용 로그 파일을 append 모드로 엽니다.
    with open('crawler_info.log','a', encoding='utf-8')as f_info_log, \
        open('crawler_error.log','a', encoding='utf-8')as f_error_log:

        # 추출 내용을 저장할 딕셔너리
        page_contents = {}

        # 터미널에 처리 시작을 출력하고, 로그 파일에도 메시지를 출력
        msg = '크롤링을 시작합니다.\n'
        print(msg)
        f_info_log.write(msg)
        
        for page_url in PAGE_URL_LIST:
            try:
                r = requests.get(page_url, timeout=30)
                r.raise_for_status() # 응답에 에러 발생시 예외를 발생시킵니다.

            except requests.exceptions.RequestException as e:
                # requests와 관련된 예외가 발생하면
                # 터미널과 오류 로그에 오류를 출력합니다. 
                msg = '[ERROR] {exception}\n'.format(exception=e)
                print(msg)
                f_error_log.write(msg)
                continue # 예외가 발생하면 반복을 중지하는게 아니라 건너 뜁니다.

            # 정상적으로 내용을 추출했다면 딕셔너리에 내용을 저장합니다.page_url
            page_contents[page_url] = r.text
            time.sleep(1)

        return page_contents


if __name__ == '__main__':
    page_contents = fetch_pages()
    f_page_contents = open('page_contents.json','w')
    with open('page_contents.json', 'w', encoding='utf8') as f_page_contents:
        json.dump(page_contents, f_page_contents, ensure_ascii=False)


크롤링을 시작합니다.

[ERROR] 404 Client Error: Not Found for url: http://example.com/1.page.html

[ERROR] 404 Client Error: Not Found for url: http://example.com/2.page.html

[ERROR] 404 Client Error: Not Found for url: http://example.com/3.page.html

